Build your Logistic Regression model

Import the libraries

hints: 

- numpy

- pandas

- LogisticRegression from sklearn.linear_model 

- StandardScaler from sklearn.preprocessing

Importing the dataset

hints:
use pandas **read_csv** method to read the csv files.

>import *train.csv* and named it as *train_dataset*
>
>import *test.csv* and names it as *test_dataset*
>
>import *gender_submission.csv* and name it as *test_labels*

Check the features for the dataset

hints:

> use .info() method
>
> train_dataset.info()

Check the data distribution for both train and test dataset.

Hints:

> use .describe() method

Check the null values for both dataset

Hints:

> use .isnull().sum() method

Feature Engineering

Drop out PassengerID, Name, Ticket, Fare and Cabin column for both datasets

hints:

> use .drop method and remember to put axis = 1 since you are dropping the column

In [ ]:
#axis = 1 means that we are dropping the column


Drop Null Embarked column record for training dataset

Hints:

>use .dropna method and axis = 0

In [ ]:
# drop the rows that embarked column have null values for train_dataset


Now fill up the age values with mean values for both datasets

hints:

> use .fillna method and value  = round(np.mean()), inplace = True

In [ ]:
#fill up the age values with mean values for both datasets


Check the sex column and update it into numerical column (Male: 0, Female: 1)

hints:

> use map method

Check the embarked column and update it into numerical column (C:0, Q:1, S:1)

hints:
 > use .map method

Get the  Survived column as target value

In [ ]:
#get the Survived column as target value for prediction


Check the current features for test dataset and test labels(target value)


Hints:

> use .info method

Check the current features for train dataset and train labels(target value)


Hints:

> use .info method

Convert the dataframes to numpy array for test dataset, test labels, train dataset, train labels

Hints:

> use .values method

In [ ]:
# convert df to numpy array


Data normalization

Pickle the StandardScaler so that we can use the same data normalization model for the web app prediction for the data normalization. 

Hints:

> import pickle library
>
> call StandardScaler and save as scaler
>
> fit the StandardScaler with the train dataset
>
> save the trained StandardScaler

In [ ]:

# save the scaler object to a file


Transform the train and test dataset using scaler.transform method

Train the model using LogisticRegression

Hints:

> call the LogisticRegression model
>
> use .fit method to train the model with train dataset and train labels

Check the score on the test data

Hints:

> use .score method with test dataset and test labels

Check the coefficients for each features

Hints:

> list(zip([features],model.coef_[0]))


In [ ]:
# Analyze the coefficients
# Coefficients are the weights of the features
# The higher the coefficient, the higher the importance of the feature
# features are Pclass, Sex, Age, SibSp, Parch, Embarked


Pickle the model

Hints:

> use .dump method

In [ ]:
# save the model using pickle


Test out the pickled_model

Hints:

> use .load method